In [1]:
import os
import sys
import re
from datetime import datetime, timedelta
from dateutil.rrule import rrule, WEEKLY, FR, SU
sys.path.append('/Users/laurenthericourt/projets/trading/trading')

import psycopg2
import pandas as pd
from pandas.io.sql import read_sql
pd.set_option('display.max_rows', 500)
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

from config.load import load_conf
from db.utils import get_uri_db
from data.candle import SYMBOLS
from indicator.trend import BollingerBands, Adx, Slope, Renko
from indicator.oscillator import Macd, Rsi, Stochastic, Obv, Atr, AwesomeOscillator
from utils.utils import get_candles

In [2]:
load_conf('../config/configuration.yaml')

In [3]:
dsn = get_uri_db()
schema = 'trading'

# Get data
## Candles

In [4]:
start_date = '2021-01-01'
end_date = '2021-02-15'

candles = get_candles(dsn, schema, start_date, end_date)

## Event

In [6]:
COUNTRY_1, COUNTRY_2 = SYMBOL.split('/')

NameError: name 'SYMBOL' is not defined

In [ ]:
with psycopg2.connect(dsn) as conn:
    sql = f'set search_path = {schema};'
    sql += '''
        SELECT date, country, name, actual_value, forecast_value, is_positive
        FROM event
        WHERE importance = 3
        AND country = ANY(%(countries)s::text[])
    '''
    event = read_sql(sql, conn, params={'countries': [COUNTRY_1, COUNTRY_2]})

In [ ]:
months = ['(jan.)', '(févr.)', '(mar)', '(avr)', '(mai)', '(juin)', '(juill.)', '(août)', '(sept.)', '(oct.)', '(nov.)', '(déc)']
regex_month = re.compile(r'|'.join(months).replace('(', '\(').replace(')', '\)').replace('.', '\.'))

periods = [' (t1)', ' (t2)', ' (t3)', ' (t4)', ' m1', ' m2', ' m3', ' m4', ' m5', ' m6', ' m7', ' m8', ' m9', ' m10', ' m11', ' m12']
regex_period = re.compile(r'|'.join(periods).replace('(', '\(').replace(')', '\)'))

In [12]:
event['processed_name'] = event['name'].str.lower()
event['processed_name'] = event['processed_name'].str.replace(regex_month, '')
event['processed_name'] = event['processed_name'].str.replace(regex_period, '')
event['processed_name'] = event['processed_name'].str.strip()

In [13]:
event['processed_name'].value_counts()[:10]

stocks de pétrole brut                              571
inscriptions hebdomadaires au chômage               571
ipc (annuel)                                        263
discours de draghi, président de la bce             209
promesses de ventes de logements (mensuel)          132
rapport jolts - nouvelles offres d'emploi           132
indice pmi non manufacturier de l'ism               132
créations d'emplois dans le secteur non agricole    132
indice pmi manufacturier de l'ism                   132
taux de chômage                                     132
Name: processed_name, dtype: int64

# Show data
## Dates to hide

In [5]:
def datetime_range(start, end, delta):
    current = start
    while current < end:
        yield current
        current += delta

def compute_datetime_to_hide(start, end):
    start_dt = datetime.strptime(start, '%Y-%m-%d')
    end_dt = datetime.strptime(end, '%Y-%m-%d')
    
    fridays = list()
    for date in rrule(WEEKLY, byweekday=FR, dtstart=start_dt, until=end_dt):
        fridays.append(date)
    sundays = list()
    for date in rrule(WEEKLY, byweekday=SU, dtstart=start_dt, until=end_dt):
        sundays.append(date)
    
    date_ranges = list()
    if fridays[0] > sundays[0]:
        del sundays[0]
    if fridays[-1] > sundays[-1]:
        del fridays[-1]
    
    datetime_to_hide = list()
    for fri, sun in zip(fridays, sundays):
        res = [dt.strftime('%Y-%m-%d %H:%M:%S') for dt in datetime_range(fri.replace(hour=22),
                                                                         sun.replace(hour=19, minute=30),
                                                                         timedelta(minutes=5))]
        datetime_to_hide.extend(res)
    return datetime_to_hide

## Methods to plot result

In [6]:
def filter_data(candles, table, symbol=None, start_date=None, end_date=None):
    mask = (candles['table'] == table)
    if symbol:
        mask = mask & (candles['symbol'] == symbol)
    if start_date and end_date:
        mask = mask & (candles['date'] >= start_date) & (candles['date'] < end_date)
    candles_to_show = candles[mask]
    return candles_to_show


def show_candle(candles):
    layout = go.Layout(
        autosize=True,
        width=1400,
        height=800,
        xaxis=go.layout.XAxis(linecolor = 'black',
                              linewidth = 1,
                              mirror = True),
        xaxis2=go.layout.XAxis(linecolor = 'black',
                               linewidth = 1,
                               mirror = True),
        yaxis=go.layout.YAxis(linecolor = 'black',
                              linewidth = 1,
                              mirror = True,
                              domain=[0, 0.2]),
        yaxis2=go.layout.YAxis(linecolor = 'black',
                               linewidth = 1,
                               mirror = True,
                               domain=[0.3, 1]),

    )
    
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
    fig.update_layout(layout, xaxis2_rangeslider_visible=False)
    fig.add_trace(go.Candlestick(x=candles['date'],
                                 open=candles['open'],
                                 high=candles['high'],
                                 low=candles['low'],
                                 close=candles['close']),
                   row=2, col=1)
    
    fig.update_yaxes(fixedrange=False)
    #fig.update_xaxes(rangebreaks=[dict(values=compute_datetime_to_hide(start_date, end_date))])

    return fig
    

## Show result

In [7]:
app = JupyterDash()

app.layout = html.Div([
    html.H1(f'Candles between {start_date} and {end_date}'),
    html.Div([
        html.Div([
            html.Label('Symbol:', className='label'),
            dcc.RadioItems(
                id='symbols_radio',
                options=[{'label': x, 'value': x} for x in SYMBOLS],
                value='EUR/USD'
            )], className='col'),
        html.Div([
            html.Label('Period:', className='label'),
            dcc.RadioItems(
                id='period_radio',
                labelStyle={'display': 'inline-block'},
                options=[{'label': '5min', 'value': 'candle'},
                         {'label': '15min', 'value': 'candle15m'},
                         {'label': '30min', 'value': 'candle30m'},
                         {'label': '1h', 'value': 'candle1h'},
                         {'label': '4h', 'value': 'candle4h'},
                         {'label': '1d', 'value': 'candle1d'}],
                value='candle1d'
            )], className='col'),
        html.Div([
            html.Label('Indicators:', className='label'),
            dcc.Checklist(
                id='indicators_checklist',
                options=[{'label': 'bollinger bands', 'value': 'bollinger_bands'},
                         {'label': 'slope', 'value': 'slope'},
                         {'label': 'adx', 'value': 'adx'},
                         {'label': 'atr', 'value': 'atr'},                         
                         {'label': 'renko', 'value': 'renko'},
                         {'label': 'macd', 'value': 'macd'},
                         {'label': 'rsi', 'value': 'rsi'},
                         {'label': 'stochastic', 'value': 'stochastic'},
                         {'label': 'obv', 'value': 'obv'},
                         {'label': 'awesome oscillator', 'value': 'ao'}],
                value=[]
            )], className='col'),
    ], className='row'),
    dcc.Graph(id='candles_graph'),
])

@app.callback(
    Output('candles_graph', 'figure'),
    Input('symbols_radio', 'value'),
    Input('period_radio', 'value'),
    Input('indicators_checklist', 'value'))
def update_figure(selected_symbol, selected_period, selected_indicators):
    filtered_data = filter_data(candles, selected_period, selected_symbol)
    fig = show_candle(filtered_data)
    if 'bollinger_bands' in selected_indicators:
        bb = BollingerBands(filtered_data, 'close')
        bb.compute()
        fig = bb.plot(fig)
    if 'macd' in selected_indicators:
        m = Macd(filtered_data, 'close')
        m.compute()
        fig = m.plot(fig)
    if 'rsi' in selected_indicators:
        r = Rsi(filtered_data, 'close')
        r.compute()
        fig = r.plot(fig)
    if 'stochastic' in selected_indicators:
        s = Stochastic(filtered_data)
        s.compute(slow=True)
        fig = s.plot(fig)
    if 'adx' in selected_indicators:
        a = Adx(filtered_data)
        a.compute()
        fig = a.plot(fig)
    if 'obv' in selected_indicators:
        o = Obv(filtered_data)
        o.compute()
        fig = o.plot(fig)
    if 'renko' in selected_indicators:
        r = Renko(filtered_data, 'close')
        r.compute()
        fig = r.plot(fig)
    if 'atr' in selected_indicators:
        a = Atr(filtered_data, 'close')
        a.compute()
        fig = a.plot(fig)
    if 'slope' in selected_indicators:
        s = Slope(filtered_data, 'close')
        s.compute(10)
        fig = s.plot(fig)
    if 'ao' in selected_indicators:
        ao = AwesomeOscillator(filtered_data)
        ao.compute()
        fig = ao.plot(fig)
    return fig

app.run_server(mode='external', port=8051)

Dash app running on http://127.0.0.1:8051/


# Test indicators

In [6]:
pd.set_option('display.max_rows', 500)

In [11]:
SYMBOL = 'EUR/USD'
candles_for_indicator = candles[(candles['symbol'] == SYMBOL) & (candles['table'] == 'candle')]
#candles.set_index('date', inplace=True)

In [8]:
adx = Adx(candles_for_indicator)
adx.compute()
candles_for_indicator['adx'] = adx.result[2]

<ipython-input-8-ad63a3bfadca>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candles_for_indicator['adx'] = adx.result[2]


In [8]:
s = Slope(candles_for_indicator, 'close')
s.compute(10)

In [12]:
candles_for_indicator = candles_for_indicator[-100:]

In [31]:
previous_candles = candles_for_indicator.shift(1)
previous_candles.columns = ['previous_' + x for x in previous_candles.columns]

In [34]:
pd.concat([candles_for_indicator, previous_candles], axis=1)

,table,symbol,date,open,close,low,high,tickqty,previous_table,previous_symbol,previous_date,previous_open,previous_close,previous_low,previous_high,previous_tickqty
38784,candle,EUR/USD,2020-12-23 15:45:00,1.22040,1.21966,1.21955,1.22043,1390,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
38792,candle,EUR/USD,2020-12-23 15:50:00,1.21966,1.21887,1.21858,1.22001,2096,candle,EUR/USD,2020-12-23 15:45:00,1.22040,1.21966,1.21955,1.22043,1390.0
38800,candle,EUR/USD,2020-12-23 15:55:00,1.21888,1.21849,1.21807,1.21913,3007,candle,EUR/USD,2020-12-23 15:50:00,1.21966,1.21887,1.21858,1.22001,2096.0
38808,candle,EUR/USD,2020-12-23 16:00:00,1.21850,1.21931,1.21850,1.21932,1671,candle,EUR/USD,2020-12-23 15:55:00,1.21888,1.21849,1.21807,1.21913,3007.0
38816,candle,EUR/USD,2020-12-23 16:05:00,1.21932,1.21957,1.21928,1.21968,769,candle,EUR/USD,2020-12-23 16:00:00,1.21850,1.21931,1.21850,1.21932,1671.0
38824,candle,EUR/USD,2020-12-23 16:10:00,1.21957,1.21959,1.21949,1.22004,1779,candle,EUR/USD,2020-12-23 16:05:00,1.21932,1.21957,1.21928,1.21968,769.0
38832,candle,EUR/USD,2020-12-23 16:15:00,1.21959,1.21936,1.21918,1.21971,997,candle,EUR/USD,2020-12-23 16:10:00,1.21957,1.21959,1.21949,1.22004,1779.0
38840,candle,EUR/USD,2020-12-23 16:20:00,1.21936,1.21984,1.21907,1.22002,1225,candle,EUR/USD,2020-12-23 16:15:00,1.21959,1.21936,1.21918,1.21971,997.0
38848,candle,EUR/USD,2020-12-23 16:25:00,1.21984,1.21995,1.21967,1.22040,1765,candle,EUR/USD,2020-12-23 16:20:00,1.21936,1.21984,1.21907,1.22002,1225.0
38856,candle,EUR/USD,2020-12-23 16:30:00,1.21996,1.21985,1.21931,1.22003,2167,candle,EUR/USD,2020-12-23 16:25:00,1.21984,1.21995,1.21967,1.22040,1765.0


In [26]:
candles_for_indicator

,table,symbol,date,open,close,low,high,tickqty
38784,candle,EUR/USD,2020-12-23 15:45:00,1.22040,1.21966,1.21955,1.22043,1390
38792,candle,EUR/USD,2020-12-23 15:50:00,1.21966,1.21887,1.21858,1.22001,2096
38800,candle,EUR/USD,2020-12-23 15:55:00,1.21888,1.21849,1.21807,1.21913,3007
38808,candle,EUR/USD,2020-12-23 16:00:00,1.21850,1.21931,1.21850,1.21932,1671
38816,candle,EUR/USD,2020-12-23 16:05:00,1.21932,1.21957,1.21928,1.21968,769
38824,candle,EUR/USD,2020-12-23 16:10:00,1.21957,1.21959,1.21949,1.22004,1779
38832,candle,EUR/USD,2020-12-23 16:15:00,1.21959,1.21936,1.21918,1.21971,997
38840,candle,EUR/USD,2020-12-23 16:20:00,1.21936,1.21984,1.21907,1.22002,1225
38848,candle,EUR/USD,2020-12-23 16:25:00,1.21984,1.21995,1.21967,1.22040,1765
38856,candle,EUR/USD,2020-12-23 16:30:00,1.21996,1.21985,1.21931,1.22003,2167


In [25]:
for row in candles_for_indicator.itertuples(index=False, name='toto'):
    print(row.table)

candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle
candle


In [23]:
candles_for_indicator.reset_index(drop=True, inplace=True)
candles_for_indicator['action'] = 0
candles_for_indicator.loc[10, 'action'] = 1
candles_for_indicator.loc[15, 'action'] = -1
candles_for_indicator.loc[20, 'action'] = -1
candles_for_indicator.loc[21, 'action'] = 1
candles_for_indicator.loc[50, 'action'] = 1
candles_for_indicator.loc[52, 'action'] = -1
candles_for_indicator.loc[60, 'action'] = 1
candles_for_indicator.loc[64, 'action'] = -1

<ipython-input-23-432260873c20>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/laurenthericourt/.pyenv/versions/3.8.6/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [31]:
candles_for_indicator['cumsum_action'] = candles_for_indicator['action'].cumsum()

<ipython-input-31-ff1ca5859993>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [33]:
mask = (candles_for_indicator['action'] != 0) & (candles_for_indicator['cumsum_action'] == 0)
candles_for_indicator.loc[mask, 'cumsum_action'] = candles_for_indicator.loc[mask, 'action'] * -1

/Users/laurenthericourt/.pyenv/versions/3.8.6/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [35]:
candles_for_indicator['ret'] = candles_for_indicator['cumsum_action'] * (candles_for_indicator['close'] - candles_for_indicator['open'])

<ipython-input-35-7c0d2d0d3f32>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [37]:
candles_for_indicator

,table,symbol,date,open,close,low,high,tickqty,adx,action,cumsum_action,ret
0,candle,EUR/USD,2010-01-29 14:45:00,1.39313,1.39250,1.39232,1.39324,481,28.641484,0,0,-0.00000
1,candle,EUR/USD,2010-01-29 14:50:00,1.39250,1.39222,1.39189,1.39269,499,29.451440,0,0,-0.00000
2,candle,EUR/USD,2010-01-29 14:55:00,1.39222,1.39156,1.39095,1.39224,661,30.918736,0,0,-0.00000
3,candle,EUR/USD,2010-01-29 15:00:00,1.39156,1.39105,1.39085,1.39199,503,32.348212,0,0,-0.00000
4,candle,EUR/USD,2010-01-29 15:05:00,1.39105,1.39062,1.39044,1.39121,490,33.943617,0,0,-0.00000
5,candle,EUR/USD,2010-01-29 15:10:00,1.39062,1.39089,1.39042,1.39140,432,35.018884,0,0,0.00000
6,candle,EUR/USD,2010-01-29 15:15:00,1.39089,1.39155,1.39080,1.39179,416,35.217481,0,0,0.00000
7,candle,EUR/USD,2010-01-29 15:20:00,1.39155,1.39177,1.39128,1.39223,506,34.578309,0,0,0.00000
8,candle,EUR/USD,2010-01-29 15:25:00,1.39177,1.39289,1.39175,1.39294,549,32.839835,0,0,0.00000
9,candle,EUR/USD,2010-01-29 15:30:00,1.39289,1.39227,1.39221,1.39305,420,31.062214,0,0,-0.00000


In [32]:
import numpy as np
pd.Series(np.where(delta < 0, delta, 0))

0       0.00000
1       0.00000
2      -0.00012
3       0.00000
4       0.00000
         ...   
9982    0.00000
9983   -0.00055
9984    0.00000
9985    0.00000
9986   -0.00348
Length: 9987, dtype: float64

In [26]:
ma, bb_up, bb_down = bollinger_bands(candles_for_indicator, 'close')

In [20]:
macd_, signal, hist = macd(candles_for_indicator, 'close')

In [21]:
hist

1           NaN
13          NaN
19          NaN
26          NaN
32          NaN
         ...   
90    -0.000371
98    -0.000097
106    0.000361
114    0.001055
122    0.001435
Name: close, Length: 6546, dtype: float64

In [29]:
bb_down

1           NaN
15          NaN
22          NaN
45          NaN
51          NaN
         ...   
170    1.178107
181    1.179303
184    1.180845
194    1.181929
206    1.182523
Name: close, Length: 9987, dtype: float64

In [11]:
type(signal)

pandas.core.series.Series

In [17]:
candles_for_indicator

NameError: name 'candles_for_indicator' is not defined

In [24]:
candles_for_indicator['max'] = candles_for_indicator[['open', 'close', 'low', 'high']].max(axis=1)